# Hitori

The rules of Hitori are simple:
You have to shade some of the cells of the grid according to the rules:
- No number should appear unshaded more than once in a row or a column.
- 2 black cells cannot be adjacent horizontally or vertically.
- All non-shaded cells should be connected in a single group by vertical or horizontal motion.

-----

中文翻译待补充。



In [3]:

import numpy as np
from ortools.sat.python import cp_model as cp
import gurobipy as gp
from collections import deque
from itertools import chain

In [4]:
def readGrid(path):
    with open(f"../assets/data/hitori/{path}.txt") as f:
        num = f.readline()
        m, n = num.split(" ")[0], num.split(" ")[1]
        grid = f.readlines()
        res = [list(map(int, g.strip().split(" "))) for g in grid]
        return int(m), int(n), res

if __name__ == "__main__":
    m, n, res = readGrid("5x5_1")
    
    print(m)
    print(n)
    print(res)

5
5
[[1, 3, 2, 5, 5], [1, 4, 5, 5, 1], [3, 5, 1, 3, 4], [3, 1, 1, 3, 2], [4, 2, 5, 1, 2]]


In [26]:

def HitoriSolver(m, n, grid) :
    
    Hitori = gp.Model("Hitori")
    Hitori.modelSense = gp.GRB.MINIMIZE
    Hitori.Params.lazyConstraints = 1
    Hitori.update()
    x = {}
    
    for i in range(m):
        for j in range(n):
            x[i, j] = Hitori.addVar(
            vtype = gp.GRB.BINARY,
            obj = 1,
            name = f"x[{i},{j}]")
            
    for i in range(m):
        for j in range(n - 1):
            Hitori.addConstr(gp.quicksum([x[i, j], x[i, j + 1]]) <= 1, name = f"r_{i}_{j}")
    for j in range(n):
        for i in range(m - 1):
            Hitori.addConstr(gp.quicksum([x[i, j], x[i + 1, j]]) <= 1, name = f"c_{i}_{j}")
                
    
    for i in range(m):
        pos_dict = dict()
        for j in range(n):
            if grid[i][j] not in pos_dict:
                pos_dict[grid[i][j]] = [(i, j)]
            else:
                pos_dict[grid[i][j]].append((i, j))
        for k, v in pos_dict.items():
            if len(v) > 1:
                Hitori.addConstr(gp.quicksum(x[subx, suby] for (subx, suby) in v) >= len(v) - 1, name = f"r{i}_{k}")
    
    for j in range(n):
        pos_dict = dict()
        for i in range(m):
            if grid[i][j] not in pos_dict:
                pos_dict[grid[i][j]] = [(i, j)]
            else:
                pos_dict[grid[i][j]].append((i, j))
        for k, v in pos_dict.items():
            if len(v) > 1:
                Hitori.addConstr(gp.quicksum(x[subx, suby] for (subx, suby) in v) >= len(v) - 1, name = f"c{j}_{k}")
    
    Hitori.setObjective(gp.quicksum(x[i, j] for i in range(m) for j in range(n)), gp.GRB.MINIMIZE)
    # Hitori.write("Hitori.lp")
    def border_elim(model, where):
        if (where == gp.GRB.Callback.MIPSOL):
            
            x_sol = model.cbGetSolution(model._x)
            curr_grid = [[0] * n for i in range(m)]
            for i in range(m):
                for j in range(n):
                    if x_sol[i, j]== 0:
                        curr_grid[i][j] = 1
                        
            def find_surrounding_black_cells(grid):
                rows, cols = len(grid), len(grid[0])
                visited = set()
                directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]  # Up, down, left, right movements
                total_white = sum(chain.from_iterable(grid))
                # Find the first unvisited white cell
                def find_unvisited_white():
                    for i in range(rows):
                        for j in range(cols):
                            if grid[i][j] == 1 and (i, j) not in visited:
                                return (i, j)
                    return None
                borders_cut = []
                cnt_white = 0
                while len(visited) < total_white:
                    start = find_unvisited_white()
                    if not start:
                        return []

                    # Use BFS to find all connected white cells and record the surrounding black cells
                    queue = deque([start])
                    surrounding_black_cells = set()  # Use a set to avoid duplicates
                    
                    while queue:
                        x, y = queue.popleft()
                        if (x, y) in visited:
                            continue
                        visited.add((x, y))
                        cnt_white += 1
                        # Check all four adjacent directions
                        for dx, dy in directions:
                            nx, ny = x + dx, y + dy
                            if 0 <= nx < rows and 0 <= ny < cols:
                                if grid[nx][ny] == 1 and (nx, ny) not in visited:
                                    queue.append((nx, ny))
                                elif grid[nx][ny] == 0:
                                    surrounding_black_cells.add((nx, ny))
                    borders_cut.append(list(surrounding_black_cells))

                if cnt_white == sum(chain.from_iterable(grid)) and len(borders_cut) == 1:
                    return []
                borders_cut.sort(key=lambda x: len(x))
                return borders_cut

            borders_cut = find_surrounding_black_cells(curr_grid)
            for cut in borders_cut:
                print("ADD Cut")
                model.cbLazy(gp.quicksum(x[subx, suby] for (subx, suby) in cut ) <= len(cut) - 1)
    
    Hitori._x = x
    Hitori.optimize(border_elim)
    # Visualize 
    
    for i in range(m):
        for j in range(n):
            if x[i, j].x > 1e-6:
                print("X", end = " ")
            else:
                print("0", end = " ")
        print()
    print()

    
if __name__ == "__main__":
    m, n, grid = readGrid("12x12_1")
    HitoriSolver(m, n, grid)
    

Set parameter LazyConstraints to value 1
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[rosetta2])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 321 rows, 144 columns and 648 nonzeros
Model fingerprint: 0x67c18528
Variable types: 0 continuous, 144 integer (144 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]
ADD Cut
ADD Cut
ADD Cut
ADD Cut
ADD Cut
ADD Cut
ADD Cut
ADD Cut
ADD Cut
Presolve removed 269 rows and 97 columns
Presolve time: 0.00s
Presolved: 52 rows, 47 columns, 104 nonzeros
Variable types: 0 continuous, 47 integer (47 binary)
ADD Cut
ADD Cut
ADD Cut
ADD Cut
ADD Cut
ADD Cut
ADD Cut
ADD Cut

Root relaxation: objective 4.000000e+01, 11 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth Int

![](../assets/figures/Hitori.png)

In [ ]:
W = 50
for i in range(1, n + 1):
    for w in range(W + 1, -1, -1 ):
        dp[w] = max(dp[w], dp[w - weight[i - 1]] + val[i - 1])

In [ ]:
for i in range(1, n + 1):
    for w in range(1, W + 1):
        for k in range(min(count[i - 1]), W // weight[i - 1] + 1):
            dp[i][w] = max(dp[i][w], dp[i - 1][w - k * weight[i - 1]] + k * val[i - 1])